model.inspect causes problems when working in model namespace because import inspect breaks things.

In [ ]:
if False:
    # Temporarily disable this test until I can recreate the model
    from pathlib import Path
    import os
    from tensorflow.keras.models import load_model

    if Path.cwd().stem == 'indl':
        os.chdir(Path.cwd().parent)

    layer_idx = 20  # [2, 6, 10, 14]
    n_steps = 100
    max_n_filters = 25

    model_file = Path.cwd() / 'data' / 'kjm_ecog' / 'converted' / 'faces_basic' / 'mv_model_full.h5'
    model = load_model(str(model_file))

    # When processing softmax classification layer,
    # second last dense layer should be converted from relu to linear.
    if (layer_idx == len(model.layers) - 1) and (model.layers[-2].activation != tf.keras.activations.linear):
        model.layers[-2].activation = tf.keras.activations.linear
        import tempfile
        # Save and load the model to actually apply the change.
        tmp_path = Path(tempfile.gettempdir()) / (next(tempfile._get_candidate_names()) + '.h5')
        try:
            model.save(str(tmp_path))
            model = load_model(str(tmp_path))
        finally:
            tmp_path.unlink()
    model.summary()

    maximizing_activations = visualize_layer(model, layer_idx, epochs=n_steps, loss_as_exclusive=True,
                                             upsampling_steps=1, upsampling_factor=1,
                                             filter_range=(0, max_n_filters),
                                             output_dim=(701, model.get_input_shape_at(0)[-1]))

    # Stitch timeseries together into one mega timeseries with NaN gaps.
    stitched_data = _stitch_filters(maximizing_activations, n=2, sort_by_activation=False)

    import matplotlib.pyplot as plt

    # Create a colour code cycler e.g. 'C0', 'C1', etc.
    from itertools import cycle
    colour_codes = map('C{}'.format, cycle(range(10)))

    plt.figure()
    for chan_ix in [15, 9, 8]:
        plt.plot(stitched_data[:, :, chan_ix], color=next(colour_codes))
    plt.show()